# Une méthode BIGM en Production

https://stackoverflow.com/questions/74727840/linear-programming-with-pulp-output-of-variables-are-none

J'essaie d'écrire un modèle pour une installation de production d'azote afin de minimiser les coûts d'électricité. L'installation peut produire de l'azote et l'injecter ou l'extraire du stockage. L'injection nécessite un peu d'électricité supplémentaire, mais le processus d'extraction ne nécessite aucune énergie. J'ai rédigé la fonction objectif suivante

fonction objectif

<img src = "https://i.stack.imgur.com/hZuf7.jpg">

dans laquelle la variable de décision phi(i,t) désigne le débit (en m3) pour les unités de production (phi1 - phi3) et pour l'injection et le soutirage du stockage (phi4 & phi5). La variable binaire a a été mise dans l'équation pour qu'une seule application de stockage (injection ou extraction) soit possible par étage t. la consommation d'électricité est une constante pour chaque unité en kWh/m3. P(t) désigne le prix de l'électricité.

Je suis en train de faire une première version du modèle avec PuLP pour m'appuyer dessus. J'ai essayé de linéariser le produit de la variable binaire et des variables continues avec la méthode du grand M. Cependant, la sortie du modèle est juste "Aucune" pour chaque variable de décision et je ne comprends pas pourquoi. On dirait qu'il ne trouve pas de solution du tout. J'ai probablement mal appliqué la méthode du grand M. Si quelqu'un pouvait m'aider ce serait très gentil. C'est aussi le premier morceau de code que j'ai écrit, donc si vous avez d'autres commentaires, n'hésitez pas à les partager.



In [2]:
# Import relevant packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import pulp as plp

# Model Creation
opt_model = plp.LpProblem(name='N2ProductionOpt')
opt_model.sense = plp.LpMinimize

# ----Input----------------------------------------------------------------------------------------------------------------------------------
# Time
set_T = list(range(0,24,1))
# Technical input
n = 3 # machine line number for machines[ASU1, ASU2, ASU3]
set_I = list(range(1, n+1))
low_cap=42000 #lower bound production capacity ASU's
max_cap = 60000 #upper bound production capacity ASU's
max_inj = max_extr = big_M = 180000 #upper bound injection/extraction capacity 
e_cons_blend = 0.25314 #electricity consumption in kWh/m3 at prod. capacity of 180.000 m3/h
e_cons_inj = 0.31045 #electricity consumption in kWh/m3 at prod. capacity of 180.000 m3/h
e_cons_extr = 0 #electricity consumption in kWh/m3
max_storage = 125e4 #max storage capacity
min_storage = 123e3 #minimal storage capacity
    
# Nitrogen demand 
n2_demand = [121548, 121453, 121537, 0, 0, 0, 200000, 115909, 108003, 103060, 100284, 99211, 99915, 103157, 102453, 
             106371, 107764, 117624, 123072, 123492, 120911, 113903, 107971, 107243]
# Electricity Prices -- DA prices 
energy_prices = [107, 105, 101, 103, 109, 138, 148, 149, 144, 135, 109, 110, 111, 113, 123, 137, 147, 163, 180, 187, 148, 139, 124, 119]

#-------------------------------------------------------------------------------------------------------------------------------------------

#----Decision variables--------------------------------------------------------------------------------------------------------------------------

# production flow of each ASU
produce = {(i, t): plp.LpVariable(cat='Continuous',
                                   lowBound=0, upBound=max_cap,    # <-- note LB change!!
                                   name="x_{0}_{1}".format(i,t)) 
             for i in set_I for t in set_T}
# binary variable to indicate ASU is "on" and must produce minimal output
ASU_on = {(i, t): plp.LpVariable(cat='Binary', name=f'ASU_on_{i}_{t}') for i in set_I for t in set_T}


# production flow of injection
inject = {t: plp.LpVariable(cat='Continuous',
                                   lowBound=0, upBound=max_inj, 
                                   name="y_{0}".format(t)) 
             for t in set_T}

# production flow of extraction
extract = {t: plp.LpVariable(cat='Continuous',
                                   lowBound=0, upBound=max_extr, 
                                   name="z_{0}".format(t)) 
             for t in set_T}

# amount of nitrogen available in storage
storage_level = {t: plp.LpVariable(cat='Continuous',
                                   lowBound=min_storage, upBound=max_storage, 
                                   name="s_{0}".format(t))
                 for t in set_T}

# # binary value which defines the utilization, i.e. extraction or injection, of the nitrogen storage; 
# storage_application = {(t): plp.LpVariable(cat='Binary',
#                                    lowBound=0, upBound=1,
#                                    name="l_{0}".format(t)) 
#              for t in set_T} 

# injection = {t: plp.LpVariable(cat='Continuous',
#                                    lowBound=0, upBound=max_extr, 
#                                    name="a_{0}".format(t)) 
#              for t in set_T}

# extraction = {t: plp.LpVariable(cat='Continuous',
#                                    lowBound=0, upBound=max_extr, 
#                                    name="b_{0}".format(t)) 
#              for t in set_T}

# Objective function:

objective = plp.lpSum((produce[i, t] * e_cons_blend + inject[t] * e_cons_inj + extract[t]*e_cons_extr) * energy_prices[t] for i in set_I for t in set_T)
opt_model.setObjective(objective)

#----Constraints-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# # Creating the binary setup of the storage utilization with the big M method
# for t in set_T:
#     opt_model += injection[t] <= storage_application[t] * big_M
#     opt_model += injection[t] >= 0
#     opt_model += injection[t] <= inject[t]
#     opt_model += injection[t] >= inject[t] - (1 - storage_application[t]) * big_M

#     opt_model += extraction[t] <= (1 - storage_application[t]) * big_M
#     opt_model += extraction[t] >= 0
#     opt_model += extraction[t] <= extract[t]
#     opt_model += extraction[t] >= extract[t] - (storage_application[t]) * big_M

# Constrain the min/max production by ASU and time period
for t in set_T:
    for i in set_I:
        # constrain the minimum -- force it to produce the min, if it is on
        opt_model += produce[i, t] >= ASU_on[i, t] * low_cap
        # constrain the maximum -- force it to be "on" if it produces anything, and limit it to max
        opt_model += produce[i, t] <= ASU_on[i, t] * max_cap

# Constraint to meet production demand - GOOD!
for t in set_T:
    opt_model += produce[1,t] + produce[2,t] + produce[3,t] - inject[t] + extract[t] >= n2_demand[t]
    
# Constraints for the nitrogen storage - GOOD!

for t in set_T:
    if t == 0:
        opt_model += storage_level[t] == max_storage + inject[t] - extract[t]
    else:
        opt_model += storage_level[t] == storage_level[t-1] + inject[t] - extract[t]
    # bounds are already set on storage variable, this is unneeded
    # opt_model += storage_level[t] >= 12.3*10**2
    # opt_model += storage_level[t] <= 36.9*10**6
  
opt_model.solve()

for t in set_T:
    print('\nFor stage {}:'.format(t))
    print('')
    for i in set_I:
        if ASU_on[i, t].varValue:   # will be true if binary var == 1
            print(f'ASU {i} running')
            print('   ASU {} flow is: {}'.format(i, produce[i, t].varValue))
         
    print('Injection flow is: {}'.format(inject[t].varValue))
    print('Extraction flow is: {}'.format(extract[t].varValue))
    print(f'storage at end of period is: {storage_level[t].varValue:0.2f}')

# print(opt_model)



For stage 0:

ASU 1 running
   ASU 1 flow is: 60000.0
ASU 3 running
   ASU 3 flow is: 60000.0
Injection flow is: 0.0
Extraction flow is: 1548.0
storage at end of period is: 1248452.00

For stage 1:

ASU 1 running
   ASU 1 flow is: 60000.0
ASU 3 running
   ASU 3 flow is: 60000.0
Injection flow is: 0.0
Extraction flow is: 1453.0
storage at end of period is: 1246999.00

For stage 2:

ASU 1 running
   ASU 1 flow is: 60000.0
ASU 3 running
   ASU 3 flow is: 60000.0
Injection flow is: 0.0
Extraction flow is: 1537.0
storage at end of period is: 1245462.00

For stage 3:

Injection flow is: 0.0
Extraction flow is: 0.0
storage at end of period is: 1245462.00

For stage 4:

Injection flow is: 0.0
Extraction flow is: 0.0
storage at end of period is: 1245462.00

For stage 5:

Injection flow is: 0.0
Extraction flow is: 0.0
storage at end of period is: 1245462.00

For stage 6:

ASU 3 running
   ASU 3 flow is: 42000.0
Injection flow is: 0.0
Extraction flow is: 158000.0
storage at end of period is: 108